In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
from qiskit_metal import Dict, designs, MetalGUI
from ThreeWaysCapacitor import TriCapacitor

# Design

In [3]:
capdesign = designs.DesignPlanar(overwrite_enabled=True)

In [4]:
gui = MetalGUI(capdesign)

In [5]:
capdesign.variables['cpw_width']='20um'
capdesign.variables['cpw_gap']='12um'

In [15]:
cap_opt = Dict(
    cap_gap='10um',
    cap_length='80um',
    cpw_length='5um'
)
central_cap = TriCapacitor(capdesign, 'CENTRAL', options=cap_opt)

gui.rebuild()
gui.zoom_on_components(['CENTRAL'])

In [16]:
#TriCapacitor.get_template_options(capdesign)
central_cap

name:    CENTRAL
class:   TriCapacitor          
options: 
  'pos_x'             : '0um',                        
  'pos_y'             : '0um',                        
  'orientation'       : '0',                          
  'chip'              : 'main',                       
  'layer'             : '1',                          
  'cap_length'        : '80um',                       
  'cap_gap'           : '10um',                       
  'cpw_width'         : '20um',                       
  'cpw_length'        : '5um',                        
  'cpw_gap'           : '12um',                       
  'gap_to_gnd'        : '12um',                       
  'hfss_wire_bonds'   : False,                        
  'q3d_wire_bonds'    : False,                        
module:  ThreeWaysCapacitor
id:      2

# Q3D

In [8]:
from qiskit_metal import renderers

In [9]:
ansys = capdesign.renderers.q3d

In [10]:
ansys.connect_ansys()

INFO 05:49PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:49PM [load_ansys_project]: 	Opened Ansys App
INFO 05:49PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 05:49PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Project9
INFO 05:49PM [connect_design]: 	Opened active design
	Design:    Sweep_Capacitance [Solution type: Q3D]
INFO 05:49PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:49PM [connect]: 	Connected to project "Project9" and design "Sweep_Capacitance" 😀 



In [35]:
ansys.activate_q3d_design('3ways_capacitor')

05:44PM 48s WARNING [activate_q3d_design]: This method is deprecated. Change your scripts to use activate_ansys_design()
INFO 05:44PM [connect_design]: 	Opened active design
	Design:    3ways_capacitor [Solution type: Q3D]


In [36]:
ansys.clean_active_design()
ansys.render_design()

In [14]:
ansys.add_q3d_setup(
    name='Capacitance',
    freq_ghz=7.2,
    min_passes=2,
    max_passes=15,
    percent_error=0.2
)

In [15]:
ansys.activate_ansys_setup('Capacitance')

INFO 05:09PM [get_setup]: 	Opened setup `Capacitance`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)


In [21]:
ansys.analyze_setup('Capacitance')

INFO 05:15PM [get_setup]: 	Opened setup `Capacitance`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:15PM [analyze]: Analyzing setup Capacitance


In [22]:
ansys.get_capacitance_matrix()

INFO 05:17PM [get_matrix]: Exporting matrix data to (D:\Users\e196203\AppData\Local\Temp\2\tmpb7x2f2yg.txt, C, , Capacitance:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 7200000000, Maxwell, 1, False


(                   cpw1_CENTRAL  cpw2_CENTRAL  cpw3_CENTRAL  ground_main_plane
 cpw1_CENTRAL           40.11240      -7.40813      -7.39559          -24.58060
 cpw2_CENTRAL           -7.40813      40.12814      -7.40118          -24.62834
 cpw3_CENTRAL           -7.39559      -7.40118      40.11758          -24.61797
 ground_main_plane     -24.58060     -24.62834     -24.61797          111.44891,
 'fF')

# Sweep

In [11]:
from qiskit_metal.analyses.sweep_and_optimize.sweeping import Sweeping

sweep = Sweeping(capdesign)

In [13]:
setup_args=Dict(
    freq_ghz=7.2,
    min_passes=2,
    max_passes=15,
    percent_error=0.2
)

all_sweep, retorn_code = sweep.sweep_one_option_get_capacitance_matrix(
    qcomp_name='CENTRAL',
    option_name='cap_length',
    option_sweep=['65um','70um','75um','80um','85um','90um'],
    qcomp_render=['CENTRAL'],
    endcaps_render=[],
    setup_args=setup_args
)

INFO 05:50PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:50PM [load_ansys_project]: 	Opened Ansys App
INFO 05:50PM [load_ansys_project]: 	Opened Ansys Desktop v2023.1.0
INFO 05:50PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    D:/Users/e196203/Documents/Ansoft/
	Project:   Project9
INFO 05:50PM [connect_design]: 	Opened active design
	Design:    Sweep_Capacitance [Solution type: Q3D]
INFO 05:50PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:50PM [connect]: 	Connected to project "Project9" and design "Sweep_Capacitance" 😀 

INFO 05:50PM [connect_design]: 	Opened active design
	Design:    Sweep_Capacitance [Solution type: Q3D]
INFO 05:50PM [get_setup]: 	Opened setup `Sweep_q3d_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:50PM [get_setup]: 	Opened setup `Sweep_q3d_setup`  (<class 'pyEPR.ansys.AnsysQ3DSetup'>)
INFO 05:50PM [analyze]: Analyzing setup Sweep_q3d_setup
INFO 05:51PM [get_matrix]: Exporting matrix data to

In [14]:
all_sweep

{'65um': {'option_name': 'cap_length',
  'convergence_target': 0.2,
  'convergence_current': 0.19389,
  'min_number_passes': 2,
  'is_convergence': True,
  'capacitance': (                   cpw1_CENTRAL  cpw2_CENTRAL  cpw3_CENTRAL  ground_main_plane
   cpw1_CENTRAL           22.19007      -3.87552      -3.87131          -14.07593
   cpw2_CENTRAL           -3.87552      22.19266      -3.86848          -14.08382
   cpw3_CENTRAL           -3.87131      -3.86848      22.19318          -14.08666
   ground_main_plane     -14.07593     -14.08382     -14.08666           77.57424,
   'fF'),
  'convergence_data':       # Triangle  Delta %
  Pass                     
  1             78      NaN
  2            236  3.00520
  3            642  3.87110
  4           1476  0.88911
  5           1876  0.41586
  6           2410  0.24522
  7           3154  0.17688
  8           4266  0.19389},
 '70um': {'option_name': 'cap_length',
  'convergence_target': 0.2,
  'convergence_current': 0.14595,
  'min

In [4]:
ansys.disconnect_ansys()

NameError: name 'ansys' is not defined